In [ ]:
from legion.metrics import send_metric, Metric
from legion.model.model_id import init

In [ ]:
init('recognize digits')

In [ ]:
# Standard scientific Python imports
import matplotlib.pyplot as plt

# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics

# The digits dataset
digits = datasets.load_digits()

# The data that we are interested in is made of 8x8 images of digits, let's
# have a look at the first 4 images, stored in the `images` attribute of the
# dataset.  If we were working from image files, we could load them using
# matplotlib.pyplot.imread.  Note that each image must have the same size. For these
# images, we know which digit they represent: it is given in the 'target' of
# the dataset.
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[:4]):
    plt.subplot(2, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %i' % label)

# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
classifier = svm.SVC(gamma=0.001)

# We learn the digits on the first half of the digits
classifier.fit(data[:n_samples // 2], digits.target[:n_samples // 2])

# Now predict the value of the digit on the second half:
expected = digits.target[n_samples // 2:]
predicted = classifier.predict(data[n_samples // 2:])

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

images_and_predictions = list(zip(digits.images[n_samples // 2:], predicted))
for index, (image, prediction) in enumerate(images_and_predictions[:4]):
    plt.subplot(2, 4, index + 5)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Prediction: %i' % prediction)

plt.show()

In [ ]:
expected = digits.target[n_samples // 2:]
predicted = classifier.predict(data[n_samples // 2:])
accuracy = metrics.accuracy_score(expected, predicted)

send_metric(Metric.TEST_ACCURACY, accuracy * 100)

In [ ]:
from PIL import Image 
import numpy as np
import pandas as pd

In [ ]:
def prepare(df):
    return df.iloc[0]

In [ ]:
def predict(v):
    image_file = v['image'].convert('L')
    image_file = image_file.resize((8, 8), Image.BOX)
    array = np.asarray(image_file, dtype=np.uint8)
    array = list(array.reshape(64))
    array = np.array([(255-x) * 1.0 / 10 for x in array], dtype=np.float)
    return {'digit': int(classifier.predict([array])[0])}
    

In [ ]:
image_file = Image.open('nine.png')

data_frame = pd.DataFrame([{'image': image_file}])

digit = predict(prepare(data_frame))
digit

In [ ]:
import legion.io
model = legion.io.export(apply_func=lambda x : predict(x),
                         use_df=False,
                         input_data_frame=data_frame,
                         version='1.0')